# Семантический анализ комментариев с BERT
Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

В распоряжении набор данных с разметкой о токсичности правок. Нужно подготовить модель классификации комментариев.

Модель должна иметь значение метрики качества F1 не меньше 0.75.

## Описание данных

`text` — текст комментария на английском

**Целевой признак**: `toxic` — метка токсичности комментария (1 - токсичный, 0 - нет)

## Последовательность шагов проекта
1. Загрузка и исследование данных.
2. Подготовка и обучение классических моделей.
3. Применение модели BERT.
4. Тестирование лучшей модели.
5. Заключение.
## 1. Загрузка и исследование данных.

In [1]:
import pkg_resources

In [2]:
try:
    pkg_resources.get_distribution('transformers')
except pkg_resources.DistributionNotFound:
    !pip3 install transformers

In [3]:
try:
    pkg_resources.get_distribution('torch')
except pkg_resources.DistributionNotFound:
    !pip install torch

In [5]:
import pandas as pd
import numpy as np
import re
from joblib import Parallel, delayed
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
import torch
import transformers
from transformers import pipeline
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

In [7]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [9]:
df.toxic.mean()

0.10161213369158527

Имеем дисбаланс классов в пользу нулевого класса.

In [10]:
df.duplicated().sum()

0

### Вывод
В исходных данных пропуски отсутствуют, дублей нет. Есть дисбаланс классов в пользу нулевого класса

## 2. Подготовка моделей
Разобьем данные на признаки и таргет

In [11]:
BATCH_SIZE = 100

In [12]:
X = df['text'].reset_index(drop=True).values.astype('U')
y = df['toxic'].reset_index(drop=True)

Создадим и применим функцию для очистки текста от всех символов, кроме латиницы.

In [13]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text) 
    text = text.split()
    return ' '.join(text)

In [14]:
cleared = []

for i in tqdm(range(X.shape[0] // BATCH_SIZE)):
    text_batch = (X[BATCH_SIZE*i:BATCH_SIZE*(i+1)]) 
    processed_texts = Parallel(n_jobs=-1)(delayed(clear_text)(t) for t in text_batch)
    cleared.append(processed_texts)

100%|██████████| 1592/1592 [00:48<00:00, 32.50it/s]


Добавим функцию лемматизации текста и применим к очищенному тексту.

In [15]:
nltk.download('averaged_perceptron_tagger')
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\epoxa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
wnl = WordNetLemmatizer()

def lemmatize(text):
    tokens = word_tokenize(text)
    return " ".join([wnl.lemmatize(token,  get_wordnet_pos(token)) for token in tokens])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\epoxa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\epoxa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\epoxa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
lemmatized = []
for batch in tqdm(cleared):
    processed_texts = Parallel(n_jobs=-1, prefer="threads")(delayed(lemmatize)(t) for t in batch)
    lemmatized += processed_texts

100%|██████████| 1592/1592 [49:56<00:00,  1.88s/it]


Сделаем тренировочную, валидационную и тестовую выборки с учетом дибаланса классов. На валидационную и тестовую выборки выделим по 20% данных

In [19]:
X = pd.Series(lemmatized)
y = y[:X.shape[0]]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, random_state=14)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=14)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_val.shape, y_val.shape

((95520,), (31840,), (95520,), (31840,), (31840,), (31840,))

Загрузим набор английских стоп-слов.

In [21]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\epoxa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2.1. Логистическая регрессия на TF-IDF.
Подготовим модель логистической регрессии на векторах TF-IDF в качестве признаков.

In [22]:
lr_pipe = Pipeline([('TF-IDF', TfidfVectorizer(stop_words=stopwords)),
                     ('logreg', LogisticRegression(max_iter=400, random_state=14, class_weight='balanced'))])
    
lr_pipe.fit(X_train, y_train)

lr_predict = lr_pipe.predict(X_val)

In [23]:
f1_score(lr_predict, y_val)

0.7466413353236532

Базовая модель немного не соответствует требованиям по качеству

### 2.2. Случайный лес на TF-IDF.
Подготовим модель случайного леса на TF-IDF

In [24]:
rf_pipe = Pipeline([('TF-IDF', TfidfVectorizer(stop_words=stopwords)),
                     ('classifier', RandomForestClassifier(class_weight='balanced'))])
    
rf_random_params = {'classifier__max_depth': list(np.arange(40, 101, step=5)),
              'classifier__n_estimators': np.arange(40, 140, step=25)
          }

rf_random = RandomizedSearchCV(rf_pipe, rf_random_params, n_iter=10, 
                               scoring='f1', n_jobs=-1, cv=3, verbose=2,
                              random_state=14)


rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('TF-IDF',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'above',
                                                                          'after',
                                                                          'again',
                                                                          'against',
                                                                          'ain',
                                                                          'all',
                                                                          'am',
                                                                          'an',
                                                                          'and',
                    

In [25]:
rf_random.best_params_, rf_random.best_score_

({'classifier__n_estimators': 65, 'classifier__max_depth': 95},
 0.5418548864060749)

In [26]:
rf_predict = rf_random.predict(X_val)

In [27]:
f1_score(rf_predict, y_val)

0.5363469761759315

### 2.3. Градиентный бустинг на TF-IDF.
Подготовим модель градиентного бустинга в реализации CatBoost на векторах TF-IDF.

In [28]:
cat_pipe = Pipeline([('TF-IDF', TfidfVectorizer(stop_words=stopwords)),
                     ('classifier', CatBoostClassifier())])

cat_pipe.fit(X_train, y_train)

Learning rate set to 0.072183
0:	learn: 0.6203362	total: 953ms	remaining: 15m 51s
1:	learn: 0.5600854	total: 1.71s	remaining: 14m 12s
2:	learn: 0.5067990	total: 2.54s	remaining: 14m 5s
3:	learn: 0.4650354	total: 3.24s	remaining: 13m 27s
4:	learn: 0.4288122	total: 3.94s	remaining: 13m 3s
5:	learn: 0.3968378	total: 4.68s	remaining: 12m 55s
6:	learn: 0.3712057	total: 5.42s	remaining: 12m 49s
7:	learn: 0.3498562	total: 6.18s	remaining: 12m 46s
8:	learn: 0.3314542	total: 6.92s	remaining: 12m 42s
9:	learn: 0.3169072	total: 7.72s	remaining: 12m 44s
10:	learn: 0.3036596	total: 8.44s	remaining: 12m 38s
11:	learn: 0.2925473	total: 9.18s	remaining: 12m 35s
12:	learn: 0.2820982	total: 10s	remaining: 12m 40s
13:	learn: 0.2741011	total: 10.8s	remaining: 12m 43s
14:	learn: 0.2674793	total: 11.6s	remaining: 12m 41s
15:	learn: 0.2617444	total: 12.3s	remaining: 12m 36s
16:	learn: 0.2566362	total: 13s	remaining: 12m 32s
17:	learn: 0.2520398	total: 13.7s	remaining: 12m 29s
18:	learn: 0.2482590	total: 14.5

Pipeline(steps=[('TF-IDF',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x0000011054FE2160>)])

In [29]:
cat_predict = cat_pipe.predict(X_val)

In [30]:
f1_score(cat_predict, y_val)

0.7458926615553121

### 2.4. BERT-embeddings.
Подготовим эмбеддинги с помощью трансформера BERT и используем их в качестве признаков для обучения классическим моделей: логистической регрессии, случайного леса и градиентного бустинга. Для моделей на эмбеддингах возьмем 20% исходного датасета с сохранением соотношения классов. Это нужно, чтобы избежать переполнения памяти при сохранении эмбеддингов.

In [31]:
X, _, y, _, = train_test_split(df['text'], df['toxic'], test_size=0.8, random_state=14, stratify=df['toxic'])

In [32]:
tokenizer = transformers.BertTokenizer(
    vocab_file="vocab.txt", do_lower_case=True)

tokenized = X.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 64

padded = np.array([x[:max_len] if len(x) > max_len else x + [0]*(max_len - len(x)) for x in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [33]:
config = transformers.BertConfig.from_json_file(
    'bert_config.json')
model = transformers.BertModel.from_pretrained(
    'pytorch_model.bin', config=config)

Some weights of the model checkpoint at pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
padded.shape

(31858, 64)

In [35]:
batch_size = 100
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

100%|██████████| 318/318 [46:42<00:00,  8.81s/it]


Так как из-за размера батча, у нас не все элементы попали в эмбеддинги, нужно размер таргета привести к соответствующему размеру

In [36]:
features = np.concatenate(embeddings)
y = y[:features.shape[0]]

In [37]:
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(features, y, test_size=0.4, random_state=14, stratify=y)
X_test_bert, X_val_bert, y_test_bert, y_val_bert = train_test_split(X_test_bert, y_test_bert, test_size=0.5, random_state=14, stratify=y_test_bert)

### 2.5. Логистическая регрессия на эмбеддингах.
Обучим и применим модель логистической регрессии на подготовленных эмбеддингах.

In [38]:
lr_model = LogisticRegression(max_iter=2000, class_weight='balanced', random_state=14)
lr_model.fit(X_train_bert, y_train_bert)
lr_pred = lr_model.predict(X_val_bert)
f1_score(y_val_bert, lr_pred)

0.3584415584415584

### 2.6. Случайный лес на эмбеддингах.
Обучим и применим модель случайного леса на эмбеддингах с учетом дисбаланса классов.

In [39]:
rf_bert_random_params = {'max_depth': list(np.arange(40, 101, step=5)),
              'n_estimators': np.arange(70, 200, step=25)
          }

rf_bert_random = RandomizedSearchCV(RandomForestClassifier(class_weight='balanced'), rf_bert_random_params, n_iter=10, 
                               scoring='f1', n_jobs=-1, cv=3, verbose=2,
                              random_state=14)


rf_bert_random.fit(X_train_bert, y_train_bert)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(class_weight='balanced'),
                   n_jobs=-1,
                   param_distributions={'max_depth': [40, 45, 50, 55, 60, 65,
                                                      70, 75, 80, 85, 90, 95,
                                                      100],
                                        'n_estimators': array([ 70,  95, 120, 145, 170, 195])},
                   random_state=14, scoring='f1', verbose=2)

In [40]:
rf_bert_pred = rf_bert_random.predict(X_val_bert)
f1_score(y_val_bert, rf_bert_pred)

0.006153846153846153

### 2.7. Градиентный бустинг на эмбеддингах.
Построим градиентный бустинг на эмбеддингах.

In [41]:
cat_bert = CatBoostClassifier()

cat_bert.fit(X_train_bert, y_train_bert)

Learning rate set to 0.036286
0:	learn: 0.6609215	total: 99.7ms	remaining: 1m 39s
1:	learn: 0.6316121	total: 192ms	remaining: 1m 36s
2:	learn: 0.6045305	total: 291ms	remaining: 1m 36s
3:	learn: 0.5796737	total: 396ms	remaining: 1m 38s
4:	learn: 0.5570545	total: 521ms	remaining: 1m 43s
5:	learn: 0.5367051	total: 614ms	remaining: 1m 41s
6:	learn: 0.5181787	total: 701ms	remaining: 1m 39s
7:	learn: 0.5011323	total: 790ms	remaining: 1m 37s
8:	learn: 0.4856842	total: 877ms	remaining: 1m 36s
9:	learn: 0.4717476	total: 969ms	remaining: 1m 35s
10:	learn: 0.4587245	total: 1.07s	remaining: 1m 35s
11:	learn: 0.4469624	total: 1.16s	remaining: 1m 35s
12:	learn: 0.4359384	total: 1.25s	remaining: 1m 34s
13:	learn: 0.4263518	total: 1.34s	remaining: 1m 34s
14:	learn: 0.4174272	total: 1.44s	remaining: 1m 34s
15:	learn: 0.4088436	total: 1.56s	remaining: 1m 35s
16:	learn: 0.4013456	total: 1.64s	remaining: 1m 35s
17:	learn: 0.3942887	total: 1.74s	remaining: 1m 34s
18:	learn: 0.3876669	total: 1.83s	remaining

In [42]:
cat_bert_pred = cat_bert.predict(X_val_bert)
f1_score(y_val_bert, cat_bert_pred)

0.1339031339031339

Результаты на эмбеддингах оказались хуже, чем на TF-IDF. Удалим переменные, которые больше не понадобятся

In [43]:
del embeddings, features, X, y, X_train_bert, X_test_bert, y_train_bert, y_test_bert, X_val_bert, y_val_bert

### Вывод
Модели, обученные на эмбеддингах части датасета плохо справляются с задачей. Вероятно, это связано с нехваткой информации в неполных данных.

## 3. BERT.
Возьмем натренированную модель-трансформер BERT из свободного доступа (https://huggingface.co/s-nlp/roberta_toxicity_classifier). Используем пайплайн для её применения без дообучения.

In [44]:
bert_pipe = pipeline(model='SkolkovoInstitute/roberta_toxicity_classifier')

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Данные на вход должны подаваться в определенном виде, поэтому преобразуем тестовые признаки.

In [45]:
X_val_bert_preproc = [{'text': str(x)[:513]} for x in X_val.to_list()]

In [46]:
validation = []
for i in tqdm(range(len(X_val_bert_preproc) // BATCH_SIZE)):
        batch = X_val_bert_preproc[BATCH_SIZE*i:BATCH_SIZE*(i+1)]      
        validation.append(bert_pipe(batch))

100%|██████████| 318/318 [51:18<00:00,  9.68s/it] 


Выходные данные помимо предсказаний содержат вероятности, поэтому нужно их очистить и преобразовать в список.

In [47]:
val_bert = []
for b in validation:
    val_bert += [int(x['label'] == 'toxic') for x in b]

Также как и в случае с эмбеддингами из-за размера батча нужно привести таргет к такому же размеру

In [48]:
y_val = y_val[:len(val_bert)]

In [49]:
f1_score(y_val, val_bert)

0.8328025477707006

### Вывод
Готовая обученная модель BERT для определения токсичности комментариев показывает результаты лучше, чем все обучаемые на наших данных модели.

## 4. Тестирование лучшей модели.
Лучшие результаты показала предобученная модель BERT. Оценим её на тестовой выборке. Для начала нужно привести тестовую выборку к соответствующему виду.

In [50]:
X_test_bert_preproc = [{'text': str(x)[:513]} for x in X_test.to_list()]
test = []
for i in tqdm(range(len(X_test_bert_preproc) // BATCH_SIZE)):
        batch = X_test_bert_preproc[BATCH_SIZE*i:BATCH_SIZE*(i+1)]      
        test.append(bert_pipe(batch))

100%|██████████| 318/318 [46:51<00:00,  8.84s/it]


In [51]:
test_bert = []
for b in test:
    test_bert += [int(x['label'] == 'toxic') for x in b]
y_test = y_test[:len(test_bert)]
f1_score(y_test, test_bert)

0.8397965670692943

### Вывод
На тестовой выборке предобученная модель BERT показала результат f1 метрики 0.84.

## Заключение
Исходные данные представляют из себя пары "текст - оценка токсичности", где 1 - токсичный комментарий, 0 - нет. Присутствует дисбаланс классов в пользу нулевого класса. приблизительное соотношение 1:9. Пропуски и дубликаты в исходных данных отсутствуют.

В качестве базовой модели была выбрана модель логистической регрессии, построенная на признаках TF-IDF с учетом дисбаланса классов. Она показала значение метрики f1 на тестовых данных 0.74, что почти соответствует требованиям по качеству (0.75). Модель градиентного бустинга показала схожие результаты. Модель случайного леса на TF-IDF показала результаты хуже (~ 0.5).

Далее в качестве признаков взяли эмбеддинги исходных текстов. Из-за ограничений по памяти взяли только 20% исходных данных, а для длины установили ограничение 64. В результате таких ограничений модели показали результаты гораздо хуже моделей на TF-IDF.

В конце была опробована предобученная модель BERT, использованная сразу без дообучения. Она показала результат f1 метрики 0.83 на валидационной выборке. Это лучший результат среди испытанных моделей. Эта модель была выбрана для оценки на тестовой выборке, где дала результат f1 = 0.84.

Таким образом, для анализа текстов предпочтительнее использовать большие предобученные специализированные модели, если есть такая возможность.